対義語のデータフレームを作る

In [1]:
!apt install aptitude
!sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8
!sudo apt-get install build-essential
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [2]:
import os

# ファイルパスを指定
file_path = '/content/drive/MyDrive/PythonProjects/トレンド予測/系統空間/combined_text2.txt'

# ファイルを開いて内容を表示
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)


# 改行で分割してリストに格納
lines = content.strip().split('\n')

# 結果を表示
for i, line in enumerate(lines):
    print(f"Line {i + 1}: {line}")

import re

# 除外する文字セット
exclude_chars = 'abcdefghijklnmopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890|()--:'

# 正規表現パターンを作成
# バックスラッシュはエスケープする必要があるため、r''形式で文字列を作成します
pattern = '[{}]'.format(re.escape(exclude_chars))

# 指定された文字を除外する関数
def remove_specified_chars(text):
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# 指定された文字を除外
cleaned_content = remove_specified_chars(content)

# 改行で分割してリストに格納
lines = cleaned_content.strip().split('\n')

import MeCab
from gensim.models import Word2Vec
import numpy as np

# MeCabの初期化
tagger = MeCab.Tagger('-Ochasen')

# 日本語のテキスト
texts = lines

# 形態素解析してトークン化
def tokenize(text):
    parsed_text = tagger.parse(text)
    tokens = [line.split('\t')[0] for line in parsed_text.splitlines() if line and not line.startswith('EOS')]
    return tokens

# トークン化したテキストをリストにする
tokenized_texts = [tokenize(text) for text in texts]

# Word2Vecモデルの訓練
model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, sg=0)

# モデルの保存
model.save("word2vec_model.bin")

# モデルの読み込み
model = Word2Vec.load("word2vec_model.bin")

# 系統名のリスト
style_names = [
    'ベーシック',
    'アメリカン',
    'モード',
    'スポーツ',
    'ストリート',
    'アウトドア',
    'エレガンス',
    'ガーリー',
    'オフィス',
    'キャリア',
    'マニッシュ',
    'デザイナーズ',
    'トラッド',
    'ノームコア',
    'ナチュラル',
    '韓国',
    '量産型',
    '地雷系'
]

# 系統名のトークン化
style_tokens = [tokenize(name) for name in style_names]

# 系統名のベクトル化
def get_mean_vector(words, model):
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 系統名リストをベクトル化
style_vectors = {name: get_mean_vector(tokenize(name), model) for name in style_names}

# 類似度計算関数
def calculate_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

# 対義語を求める関数
def find_antonym(target_word, style_names, style_vectors, model):
    if target_word not in model.wv:
        print(f"Word '{target_word}' not found in the model.")
        return None, None

    target_vector = model.wv[target_word]
    antonyms = {}
    for name in style_names:
        name_vector = style_vectors[name]
        similarity = calculate_similarity(target_vector, name_vector)
        antonyms[name] = similarity

    # 類似度が最も低い系統名を「対義語」として選ぶ
    antonym = min(antonyms, key=antonyms.get)
    return antonym, antonyms[antonym]

# 対義語の計算
list_antonym = []
for i in range(len(style_names)):
  word = style_names[i]  # 対義語を調べたい単語
  antonym, similarity = find_antonym(word, style_names, style_vectors, model)

  # 結果を表示
  if antonym:
      print(f"Antonym for '{word}' is '{antonym}' with similarity score: {similarity:.2f}")
  else:
      print(f"Word '{word}' not found in the model or no antonyms found.")
  list_antonym.append(antonym)





ストリーミング出力は最後の 5000 行に切り捨てられました。
Line 60385: ¥5,490(税込)
Line 60386: セーラーカラーサイドプリーツワンピース
Line 60387: ¥7,990(税込)
Line 60388: 袖レースアップクラシカルワンピース
Line 60389: ¥7,990(税込)
Line 60390: リボン付デコルテシアーフレアワンピース
Line 60391: ¥7,990(税込)
Line 60392: 丸襟フリルリボンワンピース
Line 60393: ¥6,990(税込)
Line 60394: 細ベルトフェミニンジャンスカ
Line 60395: ¥5,490(税込)
Line 60396: 角襟Iラインレディワンピース
Line 60397: ¥7,990(税込)
Line 60398: 丸襟ミニクラシカルワンピース
Line 60399: ¥6,990(税込)
Line 60400: ラッフルフリルボウタイワンピース
Line 60401: ¥5,990(税込)
Line 60402: バックリボンロングフレアワンピース
Line 60403: ¥6,990(税込)
Line 60404: オトナテーラードレイヤースカートワンピース
Line 60405: ¥6,990(税込)
Line 60406: ヨークレース丸襟ワンピース
Line 60407: ¥5,990(税込)
Line 60408: 選べる衿＆丈ハートケーブルニットワンピース
Line 60409: ¥5,990(税込)
Line 60410: ボウタイニットフレアワンピース
Line 60411: ¥6,990(税込)
Line 60412: パワショルセーラーワンピース
Line 60413: ¥4,990(税込)
Line 60414: 胸元リボンマーメイドキャミワンピ
Line 60415: ¥4,990(税込)
Line 60416: マーメイドスリットワンピース
Line 60417: ¥5,990(税込)
Line 60418: サイドレースアップパフワンピース
Line 60419: ¥5,990(税込)
Line 60420: 2wayフリルリボンワンピース
Line 60421: ¥4,990(税込)
Line 6042

In [3]:
from gensim.models import Word2Vec

# Word2Vecの学習
sentences = [line.split() for line in lines]
model = Word2Vec(sentences, vector_size=2, window=5, min_count=1, workers=4)

# 各系統名のベクトルを取得
word_vectors = {style: model.wv[style] for style in style_names if style in model.wv}

list_style = []
list_vector = []
# ベクトル確認
for style, vector in word_vectors.items():
    list_style.append(style)
    list_vector.append(vector)
    print(f"{style}: {vector[:10]}")  # 先頭10次元を表示


ベーシック: [-0.37983057 -0.11135811]
モード: [ 0.4254882  -0.11401176]
スポーツ: [-0.49982968  0.42182994]
ストリート: [0.3902143  0.41446668]
アウトドア: [-0.37048957 -0.10014091]
エレガンス: [ 0.500697   -0.15252957]
ガーリー: [-0.5015462   0.16029917]
オフィス: [-0.17835969 -0.25159913]
キャリア: [-0.336972  -0.0256114]
マニッシュ: [-0.19988537 -0.4417791 ]
デザイナーズ: [-0.4901903  -0.48887053]
トラッド: [0.16110697 0.38150886]
ノームコア: [ 0.3343446  -0.39404884]
ナチュラル: [0.24512595 0.11486551]
韓国: [-0.28559366  0.18876427]
量産型: [-0.44150436  0.37350634]
地雷系: [ 0.05927553 -0.09895543]


In [4]:
list_vector

[array([-0.37983057, -0.11135811], dtype=float32),
 array([ 0.4254882 , -0.11401176], dtype=float32),
 array([-0.49982968,  0.42182994], dtype=float32),
 array([0.3902143 , 0.41446668], dtype=float32),
 array([-0.37048957, -0.10014091], dtype=float32),
 array([ 0.500697  , -0.15252957], dtype=float32),
 array([-0.5015462 ,  0.16029917], dtype=float32),
 array([-0.17835969, -0.25159913], dtype=float32),
 array([-0.336972 , -0.0256114], dtype=float32),
 array([-0.19988537, -0.4417791 ], dtype=float32),
 array([-0.4901903 , -0.48887053], dtype=float32),
 array([0.16110697, 0.38150886], dtype=float32),
 array([ 0.3343446 , -0.39404884], dtype=float32),
 array([0.24512595, 0.11486551], dtype=float32),
 array([-0.28559366,  0.18876427], dtype=float32),
 array([-0.44150436,  0.37350634], dtype=float32),
 array([ 0.05927553, -0.09895543], dtype=float32)]

In [5]:
a = list(zip(list_style, list_vector))
a[1]

('モード', array([ 0.4254882 , -0.11401176], dtype=float32))

In [6]:
list_vector

[array([-0.37983057, -0.11135811], dtype=float32),
 array([ 0.4254882 , -0.11401176], dtype=float32),
 array([-0.49982968,  0.42182994], dtype=float32),
 array([0.3902143 , 0.41446668], dtype=float32),
 array([-0.37048957, -0.10014091], dtype=float32),
 array([ 0.500697  , -0.15252957], dtype=float32),
 array([-0.5015462 ,  0.16029917], dtype=float32),
 array([-0.17835969, -0.25159913], dtype=float32),
 array([-0.336972 , -0.0256114], dtype=float32),
 array([-0.19988537, -0.4417791 ], dtype=float32),
 array([-0.4901903 , -0.48887053], dtype=float32),
 array([0.16110697, 0.38150886], dtype=float32),
 array([ 0.3343446 , -0.39404884], dtype=float32),
 array([0.24512595, 0.11486551], dtype=float32),
 array([-0.28559366,  0.18876427], dtype=float32),
 array([-0.44150436,  0.37350634], dtype=float32),
 array([ 0.05927553, -0.09895543], dtype=float32)]

In [7]:
import numpy as np

# ペアを作成する関数
def create_pairs(vector_list):
    num_vectors = len(vector_list)
    pairs = []

    for i in range(num_vectors // 2):
        # i番目の要素と (num_vectors - 1 - i) 番目の要素のペアを作成
        pair = (vector_list[i], vector_list[num_vectors - 1 - i])
        pairs.append(pair)

    return pairs

# ペアを作成する
pairs = create_pairs(list_vector)


print(pairs)

[(array([-0.37983057, -0.11135811], dtype=float32), array([ 0.05927553, -0.09895543], dtype=float32)), (array([ 0.4254882 , -0.11401176], dtype=float32), array([-0.44150436,  0.37350634], dtype=float32)), (array([-0.49982968,  0.42182994], dtype=float32), array([-0.28559366,  0.18876427], dtype=float32)), (array([0.3902143 , 0.41446668], dtype=float32), array([0.24512595, 0.11486551], dtype=float32)), (array([-0.37048957, -0.10014091], dtype=float32), array([ 0.3343446 , -0.39404884], dtype=float32)), (array([ 0.500697  , -0.15252957], dtype=float32), array([0.16110697, 0.38150886], dtype=float32)), (array([-0.5015462 ,  0.16029917], dtype=float32), array([-0.4901903 , -0.48887053], dtype=float32)), (array([-0.17835969, -0.25159913], dtype=float32), array([-0.19988537, -0.4417791 ], dtype=float32))]


In [8]:
import pandas as pd

df = pd.DataFrame()
df['style'] = list_style
df['vec'] = list_vector

In [9]:
import numpy as np

# ペアを作成する関数
def create_pairs(vector_list):
    num_vectors = len(vector_list)
    pairs = []

    for i in range(num_vectors // 2):
        # i番目の要素と (num_vectors - 1 - i) 番目の要素のペアを作成
        pair = (vector_list[i], vector_list[num_vectors - 1 - i])
        pairs.append(pair)

    return pairs

# ペアを作成する
pairs = create_pairs(df['vec'])


print(pairs)

[(array([-0.37983057, -0.11135811], dtype=float32), array([ 0.05927553, -0.09895543], dtype=float32)), (array([ 0.4254882 , -0.11401176], dtype=float32), array([-0.44150436,  0.37350634], dtype=float32)), (array([-0.49982968,  0.42182994], dtype=float32), array([-0.28559366,  0.18876427], dtype=float32)), (array([0.3902143 , 0.41446668], dtype=float32), array([0.24512595, 0.11486551], dtype=float32)), (array([-0.37048957, -0.10014091], dtype=float32), array([ 0.3343446 , -0.39404884], dtype=float32)), (array([ 0.500697  , -0.15252957], dtype=float32), array([0.16110697, 0.38150886], dtype=float32)), (array([-0.5015462 ,  0.16029917], dtype=float32), array([-0.4901903 , -0.48887053], dtype=float32)), (array([-0.17835969, -0.25159913], dtype=float32), array([-0.19988537, -0.4417791 ], dtype=float32))]


In [10]:
# ペアを作成する関数
def create_pairs(styles, vectors):
    num_vectors = len(vectors)
    pairs = []

    for i in range(num_vectors // 2):
        # i番目の要素と (num_vectors - 1 - i) 番目の要素のペアを作成
        style_pair = (styles[i], styles[num_vectors - 1 - i])
        vec_pair = (vectors[i], vectors[num_vectors - 1 - i])
        pairs.append((style_pair, vec_pair))

    return pairs

# ペアを作成する
pairs = create_pairs(df['style'], df['vec'])

list_antonym = []
# 結果を表示
for style_pair, vec_pair in pairs:
    print(f"Style Pair: {style_pair}")
    print(f"Vector Pair: {vec_pair}")
    print()
    list_antonym.append(style_pair)

# ベクトルのペアが正しい順序になっているか確認
for i in range(len(list_vector) // 2):
    vec1 = list_vector[i]
    vec2 = list_vector[-(i+1)]
    pair_found = any(np.array_equal(vec1, p[1][0]) and np.array_equal(vec2, p[1][1]) for p in pairs)
    print(f"Pair {i+1} correct: {pair_found}")


Style Pair: ('ベーシック', '地雷系')
Vector Pair: (array([-0.37983057, -0.11135811], dtype=float32), array([ 0.05927553, -0.09895543], dtype=float32))

Style Pair: ('モード', '量産型')
Vector Pair: (array([ 0.4254882 , -0.11401176], dtype=float32), array([-0.44150436,  0.37350634], dtype=float32))

Style Pair: ('スポーツ', '韓国')
Vector Pair: (array([-0.49982968,  0.42182994], dtype=float32), array([-0.28559366,  0.18876427], dtype=float32))

Style Pair: ('ストリート', 'ナチュラル')
Vector Pair: (array([0.3902143 , 0.41446668], dtype=float32), array([0.24512595, 0.11486551], dtype=float32))

Style Pair: ('アウトドア', 'ノームコア')
Vector Pair: (array([-0.37048957, -0.10014091], dtype=float32), array([ 0.3343446 , -0.39404884], dtype=float32))

Style Pair: ('エレガンス', 'トラッド')
Vector Pair: (array([ 0.500697  , -0.15252957], dtype=float32), array([0.16110697, 0.38150886], dtype=float32))

Style Pair: ('ガーリー', 'デザイナーズ')
Vector Pair: (array([-0.5015462 ,  0.16029917], dtype=float32), array([-0.4901903 , -0.48887053], dtype=float3

In [11]:
list_antonym

[('ベーシック', '地雷系'),
 ('モード', '量産型'),
 ('スポーツ', '韓国'),
 ('ストリート', 'ナチュラル'),
 ('アウトドア', 'ノームコア'),
 ('エレガンス', 'トラッド'),
 ('ガーリー', 'デザイナーズ'),
 ('オフィス', 'マニッシュ')]

In [29]:
list_antonym[1][1]

'量産型'

In [30]:
list_positive = []
list_negative = []
for i in range(len(list_antonym)):
  list_positive.append(list_antonym[i][0])
  list_negative.append(list_antonym[i][1])

In [31]:
df = pd.DataFrame()
df['正'] = list_positive
df['負'] = list_negative
df

,正,負
0,ベーシック,地雷系
1,モード,量産型
2,スポーツ,韓国
3,ストリート,ナチュラル
4,アウトドア,ノームコア
5,エレガンス,トラッド
6,ガーリー,デザイナーズ
7,オフィス,マニッシュ


In [32]:
csv_filename = '/content/drive/MyDrive/PythonProjects/トレンド予測/ユニクロの画像分類/data/train_antonym.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')